In [ ]:
%pip install llama-index-vector-stores-chroma
%pip install llama-index-embeddings-huggingface
!pip install llama-index
!pip install transformers einops accelerate langchain bitsandbytes
!pip install sentence_transformers #Embedding
!pip install llama_index
!pip install llama-index-embeddings-langchain
!pip install llama-index-llms-huggingface
%pip install llama-index
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor
!pip install chromadb
%pip install llama-index-vector-stores-chroma
%pip install llama-index-embeddings-huggingface
!pip install llama-index
%pip install llama-index-retrievers-bm25
!pip install KoNLPy

In [ ]:
from llama_index.core import Settings
from llama_index.core import PromptTemplate
import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever
from konlpy.tag import Okt  # konlpy의 Okt 형태소 분석기 import
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.retrievers import QueryFusionRetriever
import chromadb
import nest_asyncio
from llama_index.core.query_engine import RetrieverQueryEngine


# Define a directory to persist index data
PERSIST_DIR = "./storage"

# 한국어 불용어 리스트
korean_stopwords = ['이', '그', '저', '있다', '하다', '것', '들', '때', '등', '에서']

        # konlpy 기반의 Okt 토크나이저를 활용하여 텍스트를 한국어로 처리하는 BM25 리트리버 설정
class KonlpyTokenizer:
    def __init__(self):
        self.tokenizer = Okt()
        
    def tokenize(self, text):
        # konlpy의 morphs 함수를 사용하여 형태소 단위로 토큰화
        tokens = self.tokenizer.morphs(text)
        # 불용어 제거
        tokens = [token for token in tokens if token not in korean_stopwords]
        return tokens


class RAGService:
    def __init__(self):
        ###MODEL 구축##################
        system_prompt="""
        <|SYSTEM|>#         
        당신은 경북대학교 정보 안내 챗봇으로, 질문에 대한 답을 해주어야합니다.
        Casual Answer로 답변 형식을 지정합니다.
        당신은 학습된 내용 안에서 질문에 답변해야합니다.
        학습된 내용이 없을 경우에는, 연결된 DB내용을 참조하도록 합니다.
        학습된 내용은 datasets항목 안에 있는 것을 뜻하며,
        DB 연결은 아직 하지 않아, DB를 참조할 경우 "내용을 찾지 못했습니다."라고 답하여야합니다.
        당신은 스스로 답변 가치를 생성할 수 있습니다.
        null이나 빈 값은 반환하지 않습니다.
        url 반환 시, <SYSTEM>을 붙이지 않습니다.

        Question: 크누큐브 사이트 알려줘
        Answer: https://knucube.knu.ac.kr/

        Question: 국어국문학과 전공 커리큘럼 사이트를 알려줘
        Answer: https://home.knu.ac.kr/HOME/knujob/sub.htm?nav_code=knu1668399325&code=dept1&dept=dept-korean
        """
        # This will wrap the default prompts that are internal to llama-index
        query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

        model= HuggingFaceLLM(
            context_window=2048,
            max_new_tokens=256,
            generate_kwargs={"temperature": 0.25, "do_sample": False},
            system_prompt=system_prompt,
            query_wrapper_prompt=query_wrapper_prompt,
            tokenizer_name="Dansoeun/Llama3-owen-Ko-3-8B-Dansoeun",
            model_name="Dansoeun/Llama3-owen-Ko-3-8B-Dansoeun",
            device_map="auto",
            tokenizer_kwargs={"max_length": 2048},
            # uncomment this if using CUDA to reduce memory usage
            # model_kwargs={"torch_dtype": torch.float16}
        )
        embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
        
        Settings.llm=model
        Settings.embed_model=embed_model
        Settings.chunk_size = 512
        ########RAG##########################
        # load documents
        documents = SimpleDirectoryReader("txt_db").load_data()
        
        # initialize node parser
        splitter = SentenceSplitter(chunk_size=512)
        
        nodes = splitter.get_nodes_from_documents(documents)

        konlpytokenizer=KonlpyTokenizer()
        
        # Korean BM25 retriever
        bm25_retriever = BM25Retriever.from_defaults(
            nodes=nodes,
            similarity_top_k=2,
            stemmer=None,  # PyStemmer 대신 직접 토크나이저에서 처리
            language=None,  # 언어 기본 처리를 비활성화
            tokenizer=KonlpyTokenizer(),  # konlpy 기반의 토크나이저
        )
        
        bm25_retriever.persist("./bm25_retriever")
        loaded_bm25_retriever = BM25Retriever.from_persist_dir("./bm25_retriever")

        docstore = SimpleDocumentStore()
        docstore.add_documents(nodes)
        
        db = chromadb.PersistentClient(path="./chroma_db")
        chroma_collection = db.get_or_create_collection("dense_vectors")
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        
        storage_context = StorageContext.from_defaults(
            docstore=docstore, vector_store=vector_store
        )
        
        #index = VectorStoreIndex(nodes=nodes, storage_context=storage_context)
        self.index=VectorStoreIndex(nodes=nodes, storage_context=storage_context)
        #nest_asyncio.apply()

        self.retriever = QueryFusionRetriever(
            [
                self.index.as_retriever(similarity_top_k=2),
                BM25Retriever.from_defaults(
                    docstore=self.index.docstore, similarity_top_k=2
                ),
            ],
            num_queries=1,
            use_async=True,
        )
    def query(self, query: str) -> str:
        query_engine = RetrieverQueryEngine(self.retriever)  
        response = query_engine.query(query)
        return str(response)

        

rag_service=RAGService()
       
query_result=rag_service.query("경북대 컴학 알려줘 ")
print(query_result)